In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 5400)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 1000)
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

### Open sirsi_load.asc file and convert to Dataframe

In [ ]:
#Open file split on Document Boundary and create list
result = list(list(l for l in e.split("\n") if l) for e in open("c:\\MyPythonScripts\\userload\\blog_sample.txt").read().split("*** DOCUMENT BOUNDARY ***"))

In [ ]:
#creates dataframe uses list to fill columns
df = pd.DataFrame(columns=['LDUSER', 'UserID', 'First_Name', 'Middle_Name', 
                           'Last_Name', 'Suffix', 'Library', 'Profile', 'User_Catagory',
                           'Addr Begin', 'Phone', 'Street', 'City/State', 'Zip', 'Email',
                           'Addr End', 'Web Auth', 'Privilege Granted','GroupID'], data=result)

In [ ]:
#drops LDUSER column
df = df.drop('LDUSER', 1)

In [ ]:
#first row of df is blank, this drops it
df = df.reindex(df.index.drop(0)).reset_index(drop=True)

In [ ]:
#drops the field names that are in each cell before the value
df['UserID'] = df['UserID'].str.replace(r'\.USER_ID\.   \|a', '')
df['First_Name'] = df['First_Name'].str.replace(r'\.USER_FIRST_NAME\.   \|a', '')
df['Middle_Name'] = df['Middle_Name'].str.replace(r'\.USER_MIDDLE_NAME\.   \|a', '')
df['Last_Name'] = df['Last_Name'].str.replace(r'\.USER_LAST_NAME\.   \|a', '')
df['Suffix'] = df['Suffix'].str.replace(r'\.USER_SUFFIX\.   \|a', '')
df['Library'] = df['Library'].str.replace(r'\.USER_LIBRARY\.   \|a', '')
df['Profile'] = df['Profile'].str.replace(r'\.USER_PROFILE\.   \|a', '')
df['User_Catagory'] = df['User_Catagory'].str.replace(r'\.USER_CATEGORY1\.   \|a', '')
df['Addr Begin'] = df['Addr Begin'].str.replace(r'\.USER_ADDR1_BEGIN\.', '')
df['Phone'] = df['Phone'].str.replace(r'\.DAYPHONE\.   \|a', '')
df['Street'] = df['Street'].str.replace(r'\.STREET\.   \|a', '')
df['City/State'] = df['City/State'].str.replace(r'\.CITY/STATE\.   \|a', '')
df['Zip'] = df['Zip'].str.replace(r'\.ZIP\.   \|a', '')
df['Email'] = df['Email'].str.replace(r'\.EMAIL\.   \|a', '')
df['Addr End'] = df['Addr End'].str.replace(r'\.USER_ADDR1_END\.', '')
df['Web Auth'] = df['Web Auth'].str.replace(r'\.USER_WEB_AUTH\.   \|a', '')
df['Privilege Granted'] = df['Privilege Granted'].str.replace(r'\.USER_PRIV_GRANTED\.   \|a', '')
df['GroupID'] = df['GroupID'].str.replace(r'\.USER_GROUP_ID\.   \|a', '')

In [ ]:
#Trim White Space
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [ ]:
#drop duplicates based on UserID
df_load = df.drop_duplicates(['UserID'])

In [ ]:
df_load

### Edit file. Code may change depending on what needs to be edited.

In [ ]:
temp_profiles = df_load[df_load['Profile']=='TEMP']

In [ ]:
df_load.loc[df_load['GroupID'] == 'Undergraduate', 'Profile'] = 'S1'
df_load.loc[df_load['GroupID'] == 'Undergraduate', 'User_Catagory'] = 'Student'

In [ ]:
df_load.loc[df_load['GroupID'] == 'Graduate', 'Profile'] = 'S2'
df_load.loc[df_load['GroupID'] == 'Graduate', 'User_Catagory'] = 'Student'

In [ ]:
profiles = df_load.groupby("Profile")['UserID'].count()

In [ ]:
profiles

### Save file to csv for sharing with others

In [ ]:
#save file to csv
#df_load.to_csv("c:\\MyPythonScripts\\userload\\sirsi_load_82020.csv", index=False) 

### Format file back into ASCII format

In [ ]:
#rename columns so they format correctly on the text file
df_load.rename(columns={'UserID':'.USER_ID.   |a', 'First_Name':'.USER_FIRST_NAME.   |a', 'Middle_Name':'.USER_MIDDLE_NAME.   |a', 'Last_Name':'.USER_LAST_NAME.   |a', 'Suffix':'.USER_SUFFIX.   |a', 'Library':'.USER_LIBRARY.   |a', 'Profile':'.USER_PROFILE.   |a', 'User_Catagory':'.USER_CATEGORY1.   |a', 'GroupID':'.USER_GROUP_ID.   |a', 'Phone':'.DAYPHONE.   |a', 'Privilege Granted':'.USER_PRIV_GRANTED.   |a', 'Web Auth':'.USER_WEB_AUTH.   |a', 'Email':'.EMAIL.   |a', 'Zip':'.ZIP.   |a', 'City/State':'.CITY/STATE.   |a', 'Street':'.STREET.   |a', 'Addr Begin':'.USER_ADDR1_BEGIN.', 'Addr End':'.USER_ADDR1_END.'},inplace=True)

In [ ]:
#add NaN to blank fields so text file formats correctly
df_load = df_load.replace('', np.NaN)

In [ ]:
#print a small amount of the file to verify format
doc = '*** DOCUMENT BOUNDARY ***'
ldu = 'FORM=LDUSER'
for i, row in df_load.head().iterrows():
        print(doc + '\n' + ldu)
        print(re.sub('([\|][a])\s+', r'\1', str(row)))        

In [ ]:
#save formated file
#use Notepad ++ to replace using regex. Replace all NaN with blank. Replace Name:\s\d+,\sdtype:\sobject with blank
doc = '*** DOCUMENT BOUNDARY ***'
ldu = 'FORM=LDUSER'
with open('c:\\MyPythonScripts\\userload\\blog_sample_new.txt', 'w') as f:
    for i, row in df_load.iterrows():
        f.write(str(doc + '\n' + ldu + '\n'))
        f.write(re.sub('([\|][a])\s+', r'\1', str(row)))
f.close()

### After export use Notepad++ to replace using regex. Replace all 'NaN' with blank, make sure to match case!. Replace 'Name:\s\d+,\sdtype:\sobject' with blank